# Simulation

## Simulate the Pheno-type data

### 1. Choose $n = 20$ genes

In [1]:
import numpy as np
import pandas as pd
import os

In [3]:
print(os.getcwd())
print(os.listdir('./data'))
file = 'data/SNP_in_200GENE_chr1.csv'
data = pd.read_csv(file, error_bad_lines=False)


e:\OneDrive\Programming\Python\CVc_in_bio_informatics
['10000SNPs.bimbam.csv', 'Chr1First2000.vcf', 'CV_errors_lmm.csv', 'CV_errors_ols.csv', 'CV_errors_ridge.csv', 'CV_errors_wls.csv', 'filtered_test.vcf', 'Phynotypes_1000GenomicsProjectSimulation.txt', 'selected_genes.csv', 'Simulated_10000_SNPs.csv', 'Simulated_SNPs.csv', 'SNP_in_200GENE_chr1 - copy.csv', 'SNP_in_200GENE_chr1.csv', 'SNP_in_20GENE_chr1.csv', 'testData.csv']


C:\Users\Archibald\AppData\Local\Temp\ipykernel_12752\2278546344.py:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv(file, error_bad_lines=False)


In [9]:
data.loc[data['POS'] == 43913576, :]

,GENE,POS,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,HG00103,HG00104,...,NA21128,NA21129,NA21130,NA21133,NA21135,NA21137,NA21141,NA21142,NA21143,NA21144
1588,ENSG00000198198,43913576,2,2,1,1,1,2,1,2,...,2,1,1,2,2,2,2,1,2,2
1589,ENSG00000229372,43913576,2,2,1,1,1,2,1,2,...,2,1,1,2,2,2,2,1,2,2


In [11]:
len(data['POS'].unique())

10291

In [3]:
print(data.shape)
data.head()

(10297, 2550)


,GENE,POS,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,HG00103,HG00104,...,NA21128,NA21129,NA21130,NA21133,NA21135,NA21137,NA21141,NA21142,NA21143,NA21144
0,ENSG00000227634,8269039,2,2,1,2,2,1,1,2,...,1,2,1,2,1,2,2,1,2,1
1,ENSG00000227634,8269341,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ENSG00000227634,8269373,2,2,1,2,2,1,1,2,...,1,2,1,2,1,2,2,1,2,1
3,ENSG00000227634,8269650,2,2,1,2,2,1,1,2,...,1,2,1,1,1,2,2,1,2,1
4,ENSG00000227634,8269712,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
data_selected = data.iloc[:10000, :]
data_selected.shape

(10000, 2550)

### Get Beta

In [9]:
# Get Beta
np.random.seed(1)
large_effect = 400
d = 10
beta_large = np.random.normal(0, np.sqrt(large_effect), d)
print(beta_large)
data_selected.loc[:d- 1,'beta'] = beta_large

n = len(data_selected)
small_effect = 2
beta_small = np.random.normal(0, np.sqrt(small_effect), n-d)
print(beta_small[:10])
data_selected.loc[d:,'beta'] = beta_small

beta = data_selected.beta
beta.shape

[ 32.48690727 -12.23512827 -10.56343505 -21.45937244  17.30815259
 -46.03077394  34.89623528 -15.22413802   6.38078192  -4.98740751]
[ 2.06773287 -2.91347893 -0.45596678 -0.54313488  1.60339212 -1.55548115
 -0.24385031 -1.24147928  0.05969925  0.82422518]


C:\Users\Archibald\AppData\Local\Temp\ipykernel_26860\3469917098.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_selected.loc[:d- 1,'beta'] = beta_large


(10000,)

In [41]:
from  sklearn.preprocessing import StandardScaler 
temp = data_selected.copy()
SNP = temp.drop(['GENE', 'POS', 'beta'], axis=1).T

SNP.head()


,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
HG00096,2,0,2,2,0,2,0,0,0,2,...,1,0,0,1,1,1,1,0,1,1
HG00097,2,0,2,2,0,2,0,0,0,2,...,1,1,1,1,1,1,2,0,1,1
HG00099,1,0,1,1,0,1,0,0,0,1,...,1,0,0,1,1,1,1,0,1,1
HG00100,2,0,2,2,0,2,0,0,0,2,...,1,0,0,1,1,1,1,0,1,1
HG00101,2,0,2,2,0,2,0,0,0,2,...,1,1,1,1,1,1,2,0,1,1


In [42]:
SNP1 = SNP.copy()

In [43]:
temp = SNP1.values @ beta
print(temp.shape)
sigma_g2 = np.var(temp, ddof = 1) # get the overall variance
h2 = 0.5 # heritability
sigma_e2 = sigma_g2 * (1 - h2)/ h2
print("The sigma_g^2 is {:.5}, and sigma_e^2 is {:.5}".format(sigma_g2, sigma_e2))

(2548,)
The sigma_g^2 is 7824.7, and sigma_e^2 is 7824.7


In [44]:
np.mean(temp**2) - np.mean(temp) **2

7821.640499236147

Add bias term

In [45]:
np.random.seed(1)
beta0 = np.random.normal(0, np.sqrt(large_effect))
print(beta0)
print(beta.shape)
# beta_ = np.append(beta, beta0)
beta_ = np.insert(beta.values, 0, beta0)
print(beta_[:20])
SNP.insert(loc=0, column='bias', value=1)
residual = np.random.normal(0, np.sqrt(sigma_e2), len(SNP))
print(SNP.values.shape)
print(beta_.shape)
SNP['Y'] = SNP1.values @ beta + residual + beta0

32.48690727326483
(10000,)
[ 32.48690727  32.48690727 -12.23512827 -10.56343505 -21.45937244
  17.30815259 -46.03077394  34.89623528 -15.22413802   6.38078192
  -4.98740751   2.06773287  -2.91347893  -0.45596678  -0.54313488
   1.60339212  -1.55548115  -0.24385031  -1.24147928   0.05969925]
(2548, 10001)
(10001,)


In [46]:
SNP.head()

,bias,0,1,2,3,4,5,6,7,8,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,Y
HG00096,1,2,0,2,2,0,2,0,0,0,...,0,0,1,1,1,1,0,1,1,14.180118
HG00097,1,2,0,2,2,0,2,0,0,0,...,1,1,1,1,1,2,0,1,1,18.910246
HG00099,1,1,0,1,1,0,1,0,0,0,...,0,0,1,1,1,1,0,1,1,11.417205
HG00100,1,2,0,2,2,0,2,0,0,0,...,0,0,1,1,1,1,0,1,1,268.919371
HG00101,1,2,0,2,2,0,2,0,0,0,...,1,1,1,1,1,2,0,1,1,-145.655915


In [25]:
# Save Simulated Parameters and data
SNP.to_csv('data/Simulated_10000_SNPs.csv')
np.savez('Parameters/simulated_parameters_10000_SNPs.npz', sigma_e2 =sigma_e2,sigma_g2 =sigma_g2, h2 = h2, beta = beta_, large_effect_terms = d)

### Save the genotype as BIMBAM file

In [54]:
BIMBAM_format = data_selected.drop(['GENE',  'beta'], axis = 1)
BIMBAM_format['POS'] = BIMBAM_format['POS'].apply(lambda x: 'pos'+str(x))
n = len(BIMBAM_format)
BIMBAM_format.insert(1, 'major', ['A']*n)
BIMBAM_format.insert(2, 'minor', ['G']*n)
BIMBAM_format.head()
BIMBAM_format.to_csv('./data/10000SNPs.bimbam.csv', sep=',', header = False, index=False)

In [53]:
phyno = SNP['Y'].to_numpy()
np.savetxt( './data/Phynotypes_1000GenomicsProjectSimulation.txt',phyno)

## Working with K-folds

In [2]:
def get_folds_indices(nfolds, n_tr):
    if nfolds <= n_tr:
        fold_size = int(np.floor(n_tr/nfolds))
        resi = n_tr % nfolds
        fold_sizes = [0] + resi * [fold_size +1] + (nfolds - resi) * [fold_size]
        indices = np.cumsum(fold_sizes)
        folds_indices = [(indices[i], indices[i + 1])  for i in range(nfolds)]

    else:
        raise Exception("Number of folds is larger than numer of samples")
    print('First 5 fold indices : {}'.format(folds_indices[:5]))
    return folds_indices
    
def getHcv_for_Kfolds(X_tr, y_tr,  H_function, V = None, nfolds = 10):
    n_tr,p = X_tr.shape
    Hcv_k = np.zeros([n_tr,n_tr])
    
    folds_indices = get_folds_indices(nfolds = nfolds, n_tr = n_tr)
    for Kindices in folds_indices:
        ia,ib = Kindices
        indices_minus_K = list(range(0, ia)) + list(range(ib, n_tr))

        X_minus_K = X_tr[indices_minus_K,:]
        y_minus_K = y_tr[indices_minus_K]
        X_k = X_tr[ia:ib,:]
        y_K = y_tr[ia:ib]

        if V is not None:
            V_minus_k = V[indices_minus_K,:][:, indices_minus_K]
            temp = H_function(X_minus_K, X_k , y_minus_K, y_K, V_minus_k)
        else:
            temp = H_function(X_minus_K, X_k , y_minus_K, y_K)

        Hcv_k[ia:ib, indices_minus_K] = temp

    return Hcv_k

In [100]:
import time
import functools

def timing(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        print('--- start function ' + func.__name__, '---')
        result = func(*args, **kwargs)
        print('------ {:.4f} seconds -----'.format(time.time() - start_time))
        return result

    return wrapper

## Model Test

### 1. LMM (linear mixed model)
Let the model be
$$
Y_i = \sum_{j = 1}^{p} \beta_j X_{i,j} + u_i +\epsilon_i \ \text{where } i = 1, 2,3,\dots,n
$$
where $(u_1, u_2, \dots, u_n)^\top \sim MVN(0, \sigma_g^2 K ) $ and $\epsilon_i \sim ^{i.i.d} N(0, \sigma_e^2)$, and  $K = 1/p X X^\top$, $\sigma_e^2 = \frac{1 - h^2}{h^2} \sigma_g^2$ . 

> ? how to estimate $\sigma_e \text{ and } \sigma_g$?  Maybe using REML? But it requires the indication of the clusters.

We can write the model as 
$$
Y_i = \sum_{j = 1}^{p} \beta_j X_{i,j} + \epsilon_i^* \ \text{where } i = 1, 2,3,\dots,n \text{ and } \boldsymbol{\epsilon}^{*} = \boldsymbol u + \boldsymbol \epsilon \sim N_{n}(\mathbf{0}, V)
$$
where $V = Var(Y|X\beta) = \sigma^2_g K + \sigma^2_e I$

So we need to estimate two of  $\sigma_g, \sigma_e \text{or } h$

so we can get estimate for $\tilde{\boldsymbol{\beta}}:=\left(X^{t} V^{-1} X\right)^{-1} X^{t} V^{-1} \boldsymbol{Y}$

BLUP for $\tilde u = E(u|Y) = \sigma^2_g K V^{-1}(Y - X\tilde \beta)$

 And $\hat Y = X \tilde \beta + \tilde u$

Now we can define $H$ by

$H = X\left(X^{t} V^{-1} X\right)^{-1} X^{t} V^{-1} + \sigma^2_g K V^{-1}(I - X\left(X^{t} V^{-1} X\right)^{-1} X^{t} V^{-1})$


$$
\widehat{C V}_{c}=\frac{1}{n}\left(\boldsymbol{y}-H_{c v} \boldsymbol{y}\right)^{t}\left(\boldsymbol{y}-H_{c v} \boldsymbol{y}\right)+\frac{2}{n}\left[\operatorname{tr}\left(H_{c v} \operatorname{Cov}(\boldsymbol{y}, \boldsymbol{y})\right)-n \boldsymbol{h}_{t e} \operatorname{Cov}\left(\boldsymbol{y}_{t r}, y_{t e}\right)\right]
$$

Get $V = Var(Y|X\beta) = \sigma^2_g K + \sigma^2_e I$ where $K = 1/p X X^\top$

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
if not 'SNP' in locals():
    SNP = pd.read_csv('data/Simulated_10000_SNPs.csv', index_col=0)
par = np.load('Parameters/simulated_parameters_10000_SNPs.npz')
sigma_e2, sigma_g2, h2, beta, num_large_effet_terms = [par[params] for params in par.files]
sigma_g2 = float(sigma_g2)
sigma_e2 = float(sigma_e2)
h2 = float(h2)
SNP.head()

,bias,0,1,2,3,4,5,6,7,8,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,Y
HG00096,1,2,0,2,2,0,2,0,0,0,...,0,0,1,1,1,1,0,1,1,14.180118
HG00097,1,2,0,2,2,0,2,0,0,0,...,1,1,1,1,1,2,0,1,1,18.910246
HG00099,1,1,0,1,1,0,1,0,0,0,...,0,0,1,1,1,1,0,1,1,11.417205
HG00100,1,2,0,2,2,0,2,0,0,0,...,0,0,1,1,1,1,0,1,1,268.919371
HG00101,1,2,0,2,2,0,2,0,0,0,...,1,1,1,1,1,2,0,1,1,-145.655915


In [12]:
SNP.shape

(2548, 10002)

In [3]:
beta[:num_large_effet_terms+1]

array([ 32.48690727,  32.48690727, -12.23512827, -10.56343505,
       -21.45937244,  17.30815259, -46.03077394,  34.89623528,
       -15.22413802,   6.38078192,  -4.98740751])

In [26]:
y = SNP['Y'].values
mu_y = np.mean(y)
sd_y = np.std(y, ddof = 1)
y = (y - mu_y) / sd_y

In [27]:
from sklearn.model_selection import train_test_split
SNP_ = SNP.copy()
SNP_.drop('Y', inplace= True, axis = 1)
data = SNP_.values
G_tr, G_te, y_tr, y_te = train_test_split(data, y, test_size = 0.2, random_state = 123)
d = 500
X_tr, X_te = G_tr[:, :d+1], G_te[:, :d+1]
W_tr, W_te = G_tr[:, 1:], G_te[:, 1:]

In [28]:
print(y_tr.shape, X_tr.shape, W_tr.shape, y_te.shape, X_te.shape, W_te.shape)

(2038,) (2038, 501) (2038, 10000) (510,) (510, 501) (510, 10000)


#### Using FAST_LMM to estimate sigma_g2 and sigma_e2

In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
# from FAST_LMM import utils as u
sc = W_tr.shape[1]
from FAST_LMM import FASTLMM
fast = FASTLMM(lowRank=True, REML = False)
fast.fit(X_tr, y_tr, 1/np.sqrt(sc) * W_tr)

------------- FAST-LMM------------------
LowRank is set as True, not using REML
Rank of W is 2038, shape of W is (2038, 10000).
--- 23.857974529266357 seconds for SVD calculation ---
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
S

In [31]:
print(fast.sigma_e2 + fast.sigma_g2)
print(np.var(y_tr), np.var(y_te), np.var(np.concatenate([y_tr, y_te])))

1.5080564670244618
0.9986777217939469 1.0029130302360851 0.9996075353218212


In [32]:
fast.summary()

---------------Summary------------------
LowRank is set as True, not using REML
Heritability h2: 0.7713843356202758
Sigma_g2: 1.1632911358935247
Sigma_e2: 0.34476533113093716


In [9]:
#using theoreatical sigma
# fast.sigma_e2 = sigma_e2
# fast.sigma_g2 = sigma_g2

print('h2 = ', 1/(1+fast.delta) )
n, sc = W_tr.shape
# varYhat = np.mean(np.diag(1/ sc * sigma_g2* W_tr @ W_tr.T + sigma_e2 * np.identity(n)))
varYhat2 = np.mean(np.diag(1/ sc * fast.sigma_g2* W_tr @ W_tr.T + fast.sigma_e2 * np.identity(n)))
# varY = np.var(y_tr - X_tr @ beta[ :num_large_effet_terms+1], ddof = 0)
varY_minus_Xbeta = np.var(y_tr - X_tr @ fast.beta)
print(varYhat2 , varY_minus_Xbeta)

h2 =  0.7713843356306709
25268.02306535538 20801.84015782223


In [10]:
n_te, sc_te = W_te.shape
y_te_hat = fast.predict(X_te,  1/np.sqrt(sc_te)* W_te)
y_tr_hat = fast.predict(X_tr, 1/np.sqrt(sc)* W_tr)

In [11]:
y_tr_hat_wls = X_tr @ fast.beta
y_te_hat_wls = X_te @ fast.beta

In [12]:
tr_error = 1/n * (np.sum(np.square(y_tr_hat - y_tr)))
te_error = 1/n_te * (np.sum(np.square(y_te_hat - y_te)))
tr_error_wls = 1/n * (np.sum(np.square(y_tr_hat_wls - y_tr)))
te_error_wls = 1/n_te * (np.sum(np.square(y_te_hat_wls - y_te)))
print('LMM: training error is:', tr_error, ' test error is:', te_error)
print('WLS: training error is:', tr_error_wls, ' test error is:', te_error_wls)


LMM: training error is: 51223228.91663165  test error is: 14240472.882552624
WLS: training error is: 44660176.62514492  test error is: 13419638.799468799


#### Comparison of training and test error of using LMM, OLS, WLS, Ridge
First we calculate $H_{tr, LMM}$, which is $H = X\left(X^{t} V^{-1} X\right)^{-1} X^{t} V^{-1} + \sigma^2_g K V^{-1}(I - X\left(X^{t} V^{-1} X\right)^{-1} X^{t} V^{-1})$

Then we calculate $H_{tr, WLS}$, which is $H = X\left(X^{t} V^{-1} X\right)^{-1} X^{t} V^{-1}$.

Finally we calculate $H_{tr, OLS}$, which is $H = X\left(X^{t} X\right)^{-1} X^{t}$.

Ridge $\hat  H_{Ridge} = X \left(X^\top X + n\lambda \mathbf{I} \right)^{-1} X^{\top} $

In [33]:
from FAST_LMM import utils as u
n_tr,sc = W_tr.shape
V_inv = fast.sigma_g2 / sc * W_tr @ W_tr.T + fast.sigma_e2 * np.identity(X_tr.shape[0])
inverse_part = u.inv(X_tr.T @ fast.V_inv() @ X_tr)

K = W_tr @ W_tr.T /sc
H_wls = X_tr @ inverse_part @ X_tr.T @ fast.V_inv()
add2 = sigma_g2 * K @ fast.V_inv() @ (np.identity(n_tr) - H_wls)
H_lmm = H_wls + add2
H_ols = X_tr @ u.inv(X_tr.T @ X_tr)@ X_tr.T

Singluar Matrix
Singluar Matrix


In [35]:
y_tr_hat_lmm = H_lmm @ y_tr
y_tr_hat_wls = H_wls @ y_tr
y_tr_hat_ols = H_ols @ y_tr 


tr_error_lmm = 1/n_tr * (np.sum((y_tr - y_tr_hat_lmm)**2))
tr_error_ols = 1/n_tr * (np.sum((y_tr - y_tr_hat_ols)**2))
tr_error_wls = 1/n_tr * (np.sum((y_tr - y_tr_hat_wls)**2))

print('Training error of lmm is {:.2f}, wls error is {:.2f}, ols error is {:.2f}, sigma_e2 is {:.2f}'.format(
    tr_error_lmm, tr_error_wls, tr_error_ols, fast.sigma_e2))
print('The ratio is {:.2f} : {:.2f} : {:.2f}'.format(1, tr_error_wls/tr_error_lmm, tr_error_ols/tr_error_lmm))

Training error of lmm is 20098523.53, wls error is 0.86, ols error is 0.67, sigma_e2 is 0.34
The ratio is 1.00 : 0.00 : 0.00


In [36]:
n_tr,sc = W_tr.shape
# V = fast.sigma_g2 / sc * W_tr @ W_tr.T + fast.sigma_e2 * np.identity(X_tr.shape[0])
inverse_part = u.inv(X_tr.T @ fast.V_inv() @ X_tr)

H_wls_te = X_te @ inverse_part @ X_tr.T @ fast.V_inv()
K_te_tr = W_te @ W_tr.T /sc
add2 = sigma_g2 * K_te_tr @ fast.V_inv() @ (np.identity(n_tr) - H_wls)
H_lmm_te = H_wls_te + add2
H_ols_te = X_te @ u.inv(X_tr.T @ X_tr)@ X_tr.T

Singluar Matrix
Singluar Matrix


In [38]:
n_te = X_te.shape[0]
y_te_hat_lmm = H_lmm_te @ y_tr
y_te_hat_wls = H_wls_te @ y_tr
y_te_hat_ols = H_ols_te @ y_tr

te_error_lmm = 1/n_te * (np.sum((y_te - y_te_hat_lmm)**2))
te_error_ols = 1/n_te * (np.sum((y_te - y_te_hat_ols)**2))
te_error_wls = 1/n_te * (np.sum((y_te - y_te_hat_wls)**2))

print('test error of lmm is {:.2f}, wls error is {:.2f}, ols error is {:.2f}, sigma_e2 is {:.2f}'.format(
    te_error_lmm, te_error_wls, te_error_ols, fast.sigma_e2))
print('The ratio is {:.2f} : {:.2f} : {:.2f}'.format(1, te_error_wls/te_error_lmm, te_error_ols/te_error_lmm))

test error of lmm is 20001956.51, wls error is 1.38, ols error is 1.55, sigma_e2 is 0.34
The ratio is 1.00 : 0.00 : 0.00


In [39]:
V_tr_te = 1/sc * W_tr @ W_te.T * sigma_g2
w_lmm = 2 *( 1/n_tr * np.trace(H_lmm @ fast.V()) - 1/ n_te * np.trace(H_lmm_te @ V_tr_te))
w_wls = 2 *( 1/n_tr * np.trace(H_wls @ fast.V()) - 1/ n_te * np.trace(H_wls_te @ V_tr_te))
w_ols = 2 *( 1/n_tr * np.trace(H_ols @ fast.V()) - 1/ n_te * np.trace(H_ols_te @ V_tr_te))


tr_error_lmm_c = tr_error_lmm + w_lmm
tr_error_wls_c = tr_error_wls + w_wls
tr_error_ols_c = tr_error_ols + w_ols

print('The training error of     lmm is {}, wls is {}, and ols is {}'.format(tr_error_lmm, tr_error_wls, tr_error_ols))
print('The corrected tr error of lmm is {}, wls is {}, and ols is {}'.format(tr_error_lmm_c, tr_error_wls_c, tr_error_ols_c))
print('The test error of         lmm is {}, wls is {}, and ols is {}'.format(te_error_lmm, te_error_wls, te_error_ols))

The training error of     lmm is 20098523.53084299, wls is 0.8630040415151261, and ols is 0.6669719582425794
The corrected tr error of lmm is -38606304.79606903, wls is -6560.636637981992, and ols is -12521.047867635652
The test error of         lmm is 20001956.508879162, wls is 1.3765027067082227, and ols is 1.5457648577533973


#### Leave One out Cross-validation error of LMM WLS and OLS

##### Using theoretical sigma values

In [40]:
summ = sigma_e2 + sigma_g2
sigma_e2 = sigma_e2/summ
sigma_g2 = sigma_g2/summ
print('sigma_e2: ', sigma_e2, ' sigma_g2: ', sigma_g2)

sigma_e2:  0.5  sigma_g2:  0.5


In [59]:
n_tr,sc = W_tr.shape
V = sigma_g2 / sc * W_tr @ W_tr.T + sigma_e2 * np.identity(n_tr)

H_cv_lmm = np.zeros([n_tr,n_tr])
H_cv_wls = np.zeros([n_tr,n_tr])
H_cv_ols = np.zeros([n_tr,n_tr])

H_cv_temp = np.zeros([n_tr,n_tr-1])
# V_minusi_inv_storage = np.zeros([n_tr,n_tr-1,n_tr-1]) 
# Store the inverse of Variance to reduce time consumpution 
# Storge comsumption will be too large

for i in range(n_tr):
    indices_mi= list(range(n_tr))
    indices_mi.remove(i)
    V_minusi_inv = u.inv(V[indices_mi,:][:,indices_mi]) 
    X_minusi = X_tr[indices_mi, :]

    inverse1 = u.inv(X_minusi.T @ V_minusi_inv @ X_minusi)
    inverse2 = u.inv(X_minusi.T @  X_minusi)

    beta_temp = inverse1 @ X_minusi.T @ V_minusi_inv
    temp = X_tr[i,:] @ beta_temp
    # H_cv_temp[i, ] = temp
    H_cv_temp = X_minusi @ beta_temp

    H_cv_wls[i, indices_mi] = temp
    H_cv_lmm[i, indices_mi] = temp + \
        V[i, indices_mi] @ u.inv(V[indices_mi,:][:,indices_mi])  @\
             (np.identity(n_tr - 1) - H_cv_temp)

    H_cv_ols[i, indices_mi] = X_tr[i,:] @ inverse2 @ X_minusi.T 

# for i in range(n_tr):
#     indices_mi= list(range(n_tr))
#     indices_mi.remove(i)
#     H_cv_lmm[i, indices_mi] = H_cv_wls[i, indices_mi] + \
#         V[i, indices_mi] @ u.inv(V[indices_mi,:][:,indices_mi])  @ (np.identity(n_tr - 1) - H_cv_temp[indices_mi,: ])

np.savez('Parameters/H_cv LMM_10000_SNPs.npz', H_cv_ols = H_cv_ols,H_cv_wls = H_cv_wls, H_cv_lmm= H_cv_lmm)


Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix


KeyboardInterrupt: 

In [82]:
np.savez('Parameters/H_cv LMM2.npz', H_cv_ols = H_cv_ols,H_cv_wls = H_cv_wls, H_cv_lmm= H_cv_lmm)

In [83]:
fast.summary()

---------------Summary------------------
LowRank is set as True, not using REML
Heritability h2: 0.6191573585757943
Sigma_g2: 0.8501751476287691
Sigma_e2: 0.5229412917597077


In [84]:
# Using Test Error to estimate the Generalization error
Error_cv_lmm = 1/n_tr * (np.sum(np.square(y_tr - H_cv_lmm @ y_tr)))
Error_cv_wls = 1/n_tr * (np.sum(np.square(y_tr - H_cv_wls @ y_tr)))
Error_cv_ols = 1/n_tr * (np.sum(np.square(y_tr - H_cv_ols @ y_tr)))



V_inv = fast.V_inv()
# using theoretical sigma_g2 to get the Covariance(y_tr, y_te)
V_tr_te = 1/sc * sigma_g2 * W_tr @ W_te.T

H_te_wls = X_te @ u.inv(X_tr.T @ V_inv @ X_tr) @ X_tr.T @ V_inv
H_te_lmm = H_te_wls + V_tr_te.T @ V_inv @ (np.identity(n_tr) - X_tr @ u.inv(X_tr.T @ V_inv @ X_tr) @ X_tr.T @ V_inv) 
H_te_ols = X_te @ u.inv(X_tr.T @ X_tr) @ X_tr.T

V = fast.V(sigma_g2 = sigma_g2, sigma_e2 = sigma_e2)
Correction_lmm = 2 * (1/n_tr * np.trace(H_cv_lmm @ V) - 1/n_te * np.trace(H_te_lmm @ V_tr_te))
Correction_wls = 2 * (1/n_tr * np.trace(H_cv_wls @ V) - 1/n_te * np.trace(H_te_wls @ V_tr_te))
Correction_ols = 2 * (1/n_tr * np.trace(H_cv_ols @ V) - 1/n_te * np.trace(H_te_ols @ V_tr_te))

Error_cv_lmm_c = Error_cv_lmm + Correction_lmm
Error_cv_wls_c = Error_cv_wls + Correction_wls
Error_cv_ols_c = Error_cv_ols + Correction_ols

Error_te_lmm = 1/n_te *(np.sum(np.square(y_te - H_te_lmm @ y_tr)))
Error_te_wls = 1/n_te *(np.sum(np.square(y_te - H_te_wls @ y_tr)))
Error_te_ols = 1/n_te *(np.sum(np.square(y_te - H_te_ols @ y_tr)))



print('The CV error of   lmm is {}, wls is {}, and ols is {}'.format(Error_cv_lmm, Error_cv_wls, Error_cv_ols))
print('The CVc error of  lmm is {}, wls is {}, and ols is {}'.format(Error_cv_lmm_c, Error_cv_wls_c, Error_cv_ols_c))
print('The test error of lmm is {}, wls is {}, and ols is {}'.format(Error_te_lmm, Error_te_wls, Error_te_ols))

The CV error of   lmm is 0.5724998389488071, wls is 0.801534662456013, and ols is 0.7889778369665089
The CVc error of  lmm is 0.868915702942935, wls is 0.8481441721079157, and ols is 0.7940647601034707
The test error of lmm is 0.6764402864058059, wls is 0.8932609152562827, and ols is 0.8572306437026114


##### Using estimated sigma value

In [62]:
sigma_g2

0.5

In [47]:
fast.summary()

---------------Summary------------------
LowRank is set as True, not using REML
Heritability h2: 0.7713843356306709
Sigma_g2: 18520.078441025507
Sigma_e2: 5488.807383553497


In [48]:
n_tr,sc = W_tr.shape
V = fast.sigma_g2 / sc * W_tr @ W_tr.T + fast.sigma_e2 * np.identity(n_tr)

H_cv_lmm2 = np.zeros([n_tr,n_tr])
H_cv_wls2 = np.zeros([n_tr,n_tr])
H_cv_ols2 = np.zeros([n_tr,n_tr])

H_cv_temp = np.zeros([n_tr,n_tr-1])
# V_minusi_inv_storage = np.zeros([n_tr,n_tr-1,n_tr-1]) 
# Store the inverse of Variance to reduce time consumpution 
# Storge comsumption will be too large

for i in range(n_tr):
    indices_mi= list(range(n_tr))
    indices_mi.remove(i)
    V_minusi_inv = u.inv(V[indices_mi,:][:,indices_mi]) 
    X_minusi = X_tr[indices_mi, :]

    inverse1 = u.inv(X_minusi.T @ V_minusi_inv @ X_minusi)
    inverse2 = u.inv(X_minusi.T @  X_minusi)

    beta_temp = inverse1 @ X_minusi.T @ V_minusi_inv
    temp = X_tr[i,:] @ beta_temp
    H_cv_temp = X_minusi @ beta_temp

    H_cv_wls2[i, indices_mi] = temp
    H_cv_lmm2[i, indices_mi] = temp + \
        V[i, indices_mi] @ u.inv(V[indices_mi,:][:,indices_mi])  @\
             (np.identity(n_tr - 1) - H_cv_temp)

    H_cv_ols2[i, indices_mi] = X_tr[i,:] @ inverse2 @ X_minusi.T 
#   H_cv_temp[i, ] = temp

# for i in range(n_tr):
#     indices_mi= list(range(n_tr))
#     indices_mi.remove(i)
#     H_cv_lmm[i, indices_mi] = H_cv_wls[i, indices_mi] + \
#         V[i, indices_mi] @ u.inv(V[indices_mi,:][:,indices_mi])  @ (np.identity(n_tr - 1) - H_cv_temp[indices_mi,: ])

np.savez('Parameters/H_cv LMM2_10000_SNPs.npz', H_cv_ols = H_cv_ols2,H_cv_wls = H_cv_wls2, H_cv_lmm= H_cv_lmm2)

Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar Matrix
Singluar

In [69]:
Hcvs = np.load('Parameters/H_cv LMM2.npz')
H_cv_ols2, H_cv_wls2, H_cv_lmm2 = [Hcvs[params] for params in Hcvs.files]

In [49]:
# Using Test Error to estimate the Generalization error
Error_cv_lmm = 1/n_tr * (np.sum(np.square(y_tr - H_cv_lmm2 @ y_tr)))
Error_cv_wls = 1/n_tr * (np.sum(np.square(y_tr - H_cv_wls2 @ y_tr)))
Error_cv_ols = 1/n_tr * (np.sum(np.square(y_tr - H_cv_ols2 @ y_tr)))



V_inv = fast.V_inv()
# using estimated sigma_g2 to get the Covariance(y_tr, y_te)
V_tr_te = 1/sc * fast.sigma_g2 * W_tr @ W_te.T

H_te_wls = X_te @ u.inv(X_tr.T @ V_inv @ X_tr) @ X_tr.T @ V_inv
H_te_lmm = H_te_wls + V_tr_te.T @ V_inv @ (np.identity(n_tr) - X_tr @ u.inv(X_tr.T @ V_inv @ X_tr) @ X_tr.T @ V_inv) 
H_te_ols = X_te @ u.inv(X_tr.T @ X_tr) @ X_tr.T

V = fast.V()
Correction_lmm = 2 * (1/n_tr * np.trace(H_cv_lmm2 @ V) - 1/n_te * np.trace(H_te_lmm @ V_tr_te))
Correction_wls = 2 * (1/n_tr * np.trace(H_cv_wls2 @ V) - 1/n_te * np.trace(H_te_wls @ V_tr_te))
Correction_ols = 2 * (1/n_tr * np.trace(H_cv_ols2 @ V) - 1/n_te * np.trace(H_te_ols @ V_tr_te))

Error_cv_lmm_c = Error_cv_lmm + Correction_lmm
Error_cv_wls_c = Error_cv_wls + Correction_wls
Error_cv_ols_c = Error_cv_ols + Correction_ols

Error_te_lmm = 1/n_te *(np.sum(np.square(y_te - H_te_lmm @ y_tr)))
Error_te_wls = 1/n_te *(np.sum(np.square(y_te - H_te_wls @ y_tr)))
Error_te_ols = 1/n_te *(np.sum(np.square(y_te - H_te_ols @ y_tr)))

print('The CV error of   lmm is {:.4f}, wls is {:.4f}, and ols is {:.4f}'.format(Error_cv_lmm, Error_cv_wls, Error_cv_ols))
print('The CVc error of  lmm is {:.4f}, wls is {:.4f}, and ols is {:.4f}'.format(Error_cv_lmm_c, Error_cv_wls_c, Error_cv_ols_c))
print('The test error of lmm is {:.4f}, wls is {:.4f}, and ols is {:.4f}'.format(Error_te_lmm, Error_te_wls, Error_te_ols))

Singluar Matrix
Singluar Matrix
Singluar Matrix
The CV error of   lmm is 13100.4556, wls is 19818.8741, and ols is 21012.8611
The CVc error of  lmm is 13281.9587, wls is 19814.8548, and ols is 21092.0336
The test error of lmm is 15450.5549, wls is 21911.6730, and ols is 24605.5762


#### Testing with random generated test data

In [41]:
np.random.seed(0)
W_te_random = np.random.choice([0,1,2], [n_te, sc])
X_te_random = np.concatenate([np.ones([n_te,1]), W_te_random[:, :d]], axis = 1)
epsilon = np.random.normal(0, np.sqrt(sigma_e2), n_te)
y_te_random = W_te_random@ beta[1:]  + beta[0] + epsilon
# y_te_random = scaler.transform(y_te_random.reshape(-1, 1))


In [42]:
# Using Test Error to estimate the Generalization error
Error_cv_lmm = 1/n_tr * (np.sum(np.square(y_tr - H_cv_lmm2 @ y_tr)))
Error_cv_wls = 1/n_tr * (np.sum(np.square(y_tr - H_cv_wls2 @ y_tr)))
Error_cv_ols = 1/n_tr * (np.sum(np.square(y_tr - H_cv_ols2 @ y_tr)))



V_inv = fast.V_inv()
# using estimated sigma_g2 to get the Covariance(y_tr, y_te)
V_tr_te = 1/sc * fast.sigma_g2 * W_tr @ W_te_random.T

H_te_wls = X_te_random @ u.inv(X_tr.T @ V_inv @ X_tr) @ X_tr.T @ V_inv
H_te_lmm = H_te_wls + V_tr_te.T @ V_inv @ (np.identity(n_tr) - X_tr @ u.inv(X_tr.T @ V_inv @ X_tr) @ X_tr.T @ V_inv) 
H_te_ols = X_te_random @ u.inv(X_tr.T @ X_tr) @ X_tr.T

V = fast.V()
Correction_lmm = 2 * (1/n_tr * np.trace(H_cv_lmm2 @ V) - 1/n_te * np.trace(H_te_lmm @ V_tr_te))
Correction_wls = 2 * (1/n_tr * np.trace(H_cv_wls2 @ V) - 1/n_te * np.trace(H_te_wls @ V_tr_te))
Correction_ols = 2 * (1/n_tr * np.trace(H_cv_ols2 @ V) - 1/n_te * np.trace(H_te_ols @ V_tr_te))

Error_cv_lmm_c = Error_cv_lmm + Correction_lmm
Error_cv_wls_c = Error_cv_wls + Correction_wls
Error_cv_ols_c = Error_cv_ols + Correction_ols

Error_te_lmm = 1/n_te *(np.sum(np.square(y_te_random - H_te_lmm @ y_tr)))
Error_te_wls = 1/n_te *(np.sum(np.square(y_te_random - H_te_wls @ y_tr)))
Error_te_ols = 1/n_te *(np.sum(np.square(y_te_random - H_te_ols @ y_tr)))



print('The CV error of   lmm is {}, wls is {}, and ols is {}'.format(Error_cv_lmm, Error_cv_wls, Error_cv_ols))
print('The CVc error of  lmm is {}, wls is {}, and ols is {}'.format(Error_cv_lmm_c, Error_cv_wls_c, Error_cv_ols_c))
print('The test error of lmm is {}, wls is {}, and ols is {}'.format(Error_te_lmm, Error_te_wls, Error_te_ols))

NameError: name 'H_cv_lmm2' is not defined

#### CV with k folds

In [43]:
from k_folds_cv_helper import *

Num processors:  12


In [44]:
def H_function_ols(X_minus_k, X_k, y_minus_k, y_k ):
    return X_k @ u.inv(X_minus_k.T @ X_minus_k) @ X_minus_k.T

H_cv_ols_k = getHcv_for_Kfolds(X_tr, y_tr, H_function_ols, nfolds = 10 )

First 5 fold indices : [(0, 204), (204, 408), (408, 612), (612, 816), (816, 1020)]
Calculating fold #  0
Singluar Matrix
Calculating fold #  1
Singluar Matrix
Calculating fold #  2
Singluar Matrix
Calculating fold #  3
Singluar Matrix
Calculating fold #  4
Singluar Matrix
Calculating fold #  5
Singluar Matrix
Calculating fold #  6
Singluar Matrix
Calculating fold #  7
Singluar Matrix
Calculating fold #  8
Singluar Matrix
Calculating fold #  9
Singluar Matrix


In [45]:
from k_folds_cv_helper import timing
@timing
def H_function_wls(X_minus_k, X_k, y_minus_k, y_k, V):
    V_inv = u.inv(V)
    inverse = u.inv(X_minus_k.T @ V_inv @ X_minus_k) 
    return X_k @ inverse @ X_minus_k.T @ V_inv

H_cv_wls_k = getHcv_for_Kfolds(X_tr, y_tr, H_function_wls, fast.V(), nfolds = 10)


First 5 fold indices : [(0, 204), (204, 408), (408, 612), (612, 816), (816, 1020)]
Calculating fold #  0
--- start function H_function_wls ---
Singluar Matrix
------ 0.6401 seconds -----
Calculating fold #  1
--- start function H_function_wls ---
Singluar Matrix
------ 0.6780 seconds -----
Calculating fold #  2
--- start function H_function_wls ---
Singluar Matrix
------ 0.5598 seconds -----
Calculating fold #  3
--- start function H_function_wls ---
Singluar Matrix
------ 0.7576 seconds -----
Calculating fold #  4
--- start function H_function_wls ---
Singluar Matrix
------ 0.4005 seconds -----
Calculating fold #  5
--- start function H_function_wls ---
Singluar Matrix
------ 0.4536 seconds -----
Calculating fold #  6
--- start function H_function_wls ---
Singluar Matrix
------ 0.4667 seconds -----
Calculating fold #  7
--- start function H_function_wls ---
Singluar Matrix
------ 0.4767 seconds -----
Calculating fold #  8
--- start function H_function_wls ---
Singluar Matrix
------ 0.

In [46]:
n_tr = X_tr.shape[0]
H_cv_lmm_k = np.zeros([n_tr, n_tr])
V_inv = fast.V_inv()
V = fast.V()
inverse = u.inv (X_tr.T @ V_inv @ X_tr)
H_cv_temp = X_tr @ inverse @ X_tr.T @ V_inv

folds_indices = get_folds_indices(10, n_tr)
# get H_temp
for Kindices in folds_indices:
    ia,ib = Kindices
    indices_minus_K = list(range(0, ia)) + list(range(ib, n_tr))

    V_minus_k = V[indices_minus_K,:][:, indices_minus_K]
    X_k = X_tr[ia:ib,:]
    X_minus_k = X_tr[indices_minus_K,]

    V_inv = u.inv(V_minus_k)

    H_cv_temp = X_minus_k @ inverse @ X_minus_k.T @ V_inv

    # inverse = u.inv(X_minus_k.T @ V_inv @ X_minus_k) 

    # temp =  X_k @ inverse @ X_minus_k.T @ V_inv
    # H_cv_temp.append(temp)
    
    temp_u= V[ia:ib, indices_minus_K] @ u.inv(V_minus_k) @(
        np.identity(n_tr - (ib - ia)) - H_cv_temp #[indices_minus_K,:][:, indices_minus_K]
    )

    H_cv_lmm_k[ia:ib, indices_minus_K] = H_cv_wls_k[ia:ib, indices_minus_K] + temp_u


Singluar Matrix
First 5 fold indices : [(0, 204), (204, 408), (408, 612), (612, 816), (816, 1020)]


In [96]:
lamb = 10
@timing
def H_function_ridge(X_minus_k, X_k, y_minus_k, y_k ):
    return X_k @ u.inv(X_minus_k.T @ X_minus_k + lamb * np.identity(X_minus_k.shape[1])) @ X_minus_k.T

H_cv_ridge_k = getHcv_for_Kfolds(X_tr, y_tr, H_function_ridge, nfolds = 10)

First 5 fold indices : [(0, 204), (204, 408), (408, 612), (612, 816), (816, 1020)]
Calculating fold #  0
--- start function H_function_ridge ---
------ 1.2157 seconds -----
Calculating fold #  1
--- start function H_function_ridge ---
------ 1.2918 seconds -----
Calculating fold #  2
--- start function H_function_ridge ---
------ 1.0320 seconds -----
Calculating fold #  3
--- start function H_function_ridge ---
------ 1.2397 seconds -----
Calculating fold #  4
--- start function H_function_ridge ---
------ 1.3979 seconds -----
Calculating fold #  5
--- start function H_function_ridge ---
------ 1.0028 seconds -----
Calculating fold #  6
--- start function H_function_ridge ---
------ 1.3843 seconds -----
Calculating fold #  7
--- start function H_function_ridge ---
------ 1.3542 seconds -----
Calculating fold #  8
--- start function H_function_ridge ---
------ 1.0432 seconds -----
Calculating fold #  9
--- start function H_function_ridge ---
------ 1.0217 seconds -----


In [94]:
np.random.seed(0)
W_te_random = np.random.choice([0,1,2], [n_te, sc])
X_te_random = np.concatenate([np.ones([n_te,1]), W_te_random[:, :d]], axis = 1)

epsilon = np.random.normal(0, np.sqrt(sigma_e2), n_te)

samples = np.random.choice(np.arange(n_tr), n_te)
W_te_sample = W_tr[samples, :]
X_te_sample = X_tr[samples, :]

y_te_sample = y_tr[samples]


In [99]:
n_tr = X_tr.shape[0]
n_te = X_te.shape[0]
# Using Test Error to estimate the Generalization error
Error_cv_lmm = 1/n_tr * (np.sum(np.square(y_tr - H_cv_lmm_k @ y_tr)))
Error_cv_wls = 1/n_tr * (np.sum(np.square(y_tr - H_cv_wls_k @ y_tr)))
Error_cv_ols = 1/n_tr * (np.sum(np.square(y_tr - H_cv_ols_k @ y_tr)))
Error_cv_ridge = 1/n_tr * (np.sum(np.square(y_tr - H_cv_ridge_k @ y_tr)))



V_inv = fast.V_inv()
# using estimated sigma_g2 to get the Covariance(y_tr, y_te)
V_tr_te = 1/sc * fast.sigma_g2 * W_tr @ W_te.T

H_te_wls = X_te @ u.inv(X_tr.T @ V_inv @ X_tr) @ X_tr.T @ V_inv
H_te_lmm = H_te_wls + V_tr_te.T @ V_inv @ (np.identity(n_tr) - X_tr @ u.inv(X_tr.T @ V_inv @ X_tr) @ X_tr.T @ V_inv) 
H_te_ols = X_te @ u.inv(X_tr.T @ X_tr) @ X_tr.T
H_te_ridge = X_te @ u.inv(X_tr.T @ X_tr + lamb * np.identity(X_tr.shape[1])) @ X_tr.T

Error_te_lmm = 1/n_te *(np.sum(np.square(y_te - H_te_lmm @ y_tr)))
Error_te_wls = 1/n_te *(np.sum(np.square(y_te - H_te_wls @ y_tr)))
Error_te_ols = 1/n_te *(np.sum(np.square(y_te - H_te_ols @ y_tr)))
Error_te_ridge = 1/n_te *(np.sum(np.square(y_te - H_te_ridge @ y_tr)))

V = fast.V()
Correction_lmm = 2 * (1/n_tr * np.trace(H_cv_lmm_k @ V) - 1/n_te * np.trace(H_te_lmm @ V_tr_te))
Correction_wls = 2 * (1/n_tr * np.trace(H_cv_wls_k @ V) - 1/n_te * np.trace(H_te_wls @ V_tr_te))
Correction_ols = 2 * (1/n_tr * np.trace(H_cv_ols_k @ V) - 1/n_te * np.trace(H_te_ols @ V_tr_te))
Correction_ridge = 2 * (1/n_tr * np.trace(H_cv_ols_k @ V) - 1/n_te * np.trace(H_te_ols @ V_tr_te))



Error_cv_lmm_c = Error_cv_lmm + Correction_lmm
Error_cv_wls_c = Error_cv_wls + Correction_wls
Error_cv_ols_c = Error_cv_ols + Correction_ols
Error_cv_ridge_c = Error_cv_ridge + Correction_ols

# using  te_generated to correct
V_tr_te_random = 1/sc * fast.sigma_g2 * W_tr @ W_te_random.T

H_te_wls_r = X_te_random @ u.inv(X_tr.T @ V_inv @ X_tr) @ X_tr.T @ V_inv
H_te_lmm_r = H_te_wls_r + V_tr_te_random.T @ V_inv @ (np.identity(n_tr) - X_tr @ u.inv(X_tr.T @ V_inv @ X_tr) @ X_tr.T @ V_inv) 
H_te_ols_r = X_te_random @ u.inv(X_tr.T @ X_tr) @ X_tr.T
H_te_ridge_r = X_te_random @ u.inv(X_tr.T @ X_tr + lamb * np.identity(X_tr.shape[1])) @ X_tr.T

Correction_lmm_r = 2 * (1/n_tr * np.trace(H_cv_lmm_k @ V) - 1/n_te * np.trace(H_te_lmm_r @ V_tr_te_random))
Correction_wls_r = 2 * (1/n_tr * np.trace(H_cv_wls_k @ V) - 1/n_te * np.trace(H_te_wls_r @ V_tr_te_random))
Correction_ols_r = 2 * (1/n_tr * np.trace(H_cv_ols_k @ V) - 1/n_te * np.trace(H_te_ols_r @ V_tr_te_random))
Correction_ridge_r = 2 * (1/n_tr * np.trace(H_cv_ols_k @ V) - 1/n_te * np.trace(H_te_ridge_r @ V_tr_te_random))

Error_cv_lmm_c_r = Error_cv_lmm + Correction_lmm_r
Error_cv_wls_c_r = Error_cv_wls + Correction_wls_r
Error_cv_ols_c_r = Error_cv_ols + Correction_ols_r
Error_cv_ridge_c_r = Error_cv_ridge + Correction_ols_r


# using  sampled from trainint data to correct
V_tr_te_sample = 1/sc * fast.sigma_g2 * W_tr @ W_te_sample.T

H_te_wls_s = X_te_sample @ u.inv(X_tr.T @ V_inv @ X_tr) @ X_tr.T @ V_inv
H_te_lmm_s = H_te_wls_s + V_tr_te_sample.T @ V_inv @ (np.identity(n_tr) - X_tr @ u.inv(X_tr.T @ V_inv @ X_tr) @ X_tr.T @ V_inv) 
H_te_ols_s = X_te_sample @ u.inv(X_tr.T @ X_tr) @ X_tr.T
H_te_ridge_s = X_te_sample @ u.inv(X_tr.T @ X_tr + lamb * np.identity(X_tr.shape[1])) @ X_tr.T

Correction_lmm_s = 2 * (1/n_tr * np.trace(H_cv_lmm_k @ V) - 1/n_te * np.trace(H_te_lmm_s @ V_tr_te_sample))
Correction_wls_s = 2 * (1/n_tr * np.trace(H_cv_wls_k @ V) - 1/n_te * np.trace(H_te_wls_s @ V_tr_te_sample))
Correction_ols_s = 2 * (1/n_tr * np.trace(H_cv_ols_k @ V) - 1/n_te * np.trace(H_te_ols_s @ V_tr_te_sample))
Correction_ridge_s = 2 * (1/n_tr * np.trace(H_cv_ols_k @ V) - 1/n_te * np.trace(H_te_ridge_s @ V_tr_te_sample))

Error_cv_lmm_c_s = Error_cv_lmm + Correction_lmm_s
Error_cv_wls_c_s = Error_cv_wls + Correction_wls_s
Error_cv_ols_c_s = Error_cv_ols + Correction_ols_s
Error_cv_ridge_c_s = Error_cv_ridge + Correction_ols_s


print('The CV error of   lmm is {:.4f}, wls is {:.4f}, and ols is {:.4f}, and ridge is {:.4f}'.format(Error_cv_lmm, Error_cv_wls, Error_cv_ols, Error_cv_ridge ))
print('The CVc error of  lmm is {:.4f}, wls is {:.4f}, and ols is {:.4f}, and ridge is {:.4f}'.format(Error_cv_lmm_c, Error_cv_wls_c, Error_cv_ols_c, Error_cv_ridge_c ))
print('The CVc random of  lmm is {:.4f}, wls is {:.4f}, and ols is {:.4f}, and ridge is {:.4f}'.format(Error_cv_lmm_c_r, Error_cv_wls_c_r, Error_cv_ols_c_r, Error_cv_ridge_c_r))
print('The CVc sample of  lmm is {:.4f}, wls is {:.4f}, and ols is {:.4f}, and ridge is {:.4f}'.format(Error_cv_lmm_c_s, Error_cv_wls_c_s, Error_cv_ols_c_s, Error_cv_ridge_c_s))
print('The test error of lmm is {:.4f}, wls is {:.4f}, and ols is {:.4f}, and ridge is {:.4f}'.format(Error_te_lmm, Error_te_wls, Error_te_ols, Error_te_ridge))


The CV error of   lmm is 0.8624, wls is 1.2714, and ols is 1.3439, and ridge is 0.9775
The CVc error of  lmm is 0.8971, wls is 1.3033, and ols is 1.3477, and ridge is 0.9813
The CVc random of  lmm is 1.6779, wls is 1.8381, and ols is 3.3164, and ridge is 2.9499
The CVc sample of  lmm is 0.8105, wls is 1.2532, and ols is 1.2116, and ridge is 0.8452
The test error of lmm is 0.9705, wls is 1.3765, and ols is 1.5458, and ridge is 1.0497


In [89]:
np.random.seed(0)
W_te_random = np.random.choice([0,1,2], [n_te, sc])
X_te_random = np.concatenate([np.ones([n_te,1]), W_te_random[:, :d]], axis = 1)

epsilon = np.random.normal(0, np.sqrt(sigma_e2), n_te)
y_te_random = W_te_random@ beta[1:]  + beta[0] + epsilon
y_te_random = (y_te_random - mu_y)/sd_y

samples = np.random.choice(np.arange(n_tr), n_te)
W_te_sample = W_tr[samples, :]
X_te_sample = X_tr[samples, :]

y_te_sample = y_tr[samples]


In [91]:

Error_te_lmm_r = 1/n_te *(np.sum(np.square(y_te_random - H_te_lmm_r @ y_tr)))
Error_te_wls_r = 1/n_te *(np.sum(np.square(y_te_random - H_te_wls_r @ y_tr)))
Error_te_ols_r = 1/n_te *(np.sum(np.square(y_te_random - H_te_ols_r @ y_tr)))
Error_te_ridge_r = 1/n_te *(np.sum(np.square(y_te_random - H_te_ridge_r @ y_tr)))

Error_te_lmm_s = 1/n_te *(np.sum(np.square(y_te_sample - H_te_lmm_s @ y_tr)))
Error_te_wls_s = 1/n_te *(np.sum(np.square(y_te_sample - H_te_wls_s @ y_tr)))
Error_te_ols_s = 1/n_te *(np.sum(np.square(y_te_sample - H_te_ols_s @ y_tr)))
Error_te_ridge_s = 1/n_te *(np.sum(np.square(y_te_sample - H_te_ridge_s @ y_tr)))

print('The test error of lmm is {:.4f}, wls is {:.4f}, and ols is {:.4f}, and ridge is {:.4f}'.format(Error_te_lmm_r, Error_te_wls_r, Error_te_ols_r, Error_te_ridge_r))
print('The test error of lmm is {:.4f}, wls is {:.4f}, and ols is {:.4f}, and ridge is {:.4f}'.format(Error_te_lmm_s, Error_te_wls_s, Error_te_ols_s, Error_te_ridge_s))

The test error of lmm is 265.107913, wls is 267.231833, and ols is 386.006119, and ridge is 386.006119
The test error of lmm is 1.684299, wls is 1.535457, and ols is 1.464134, and ridge is 1.464134


### Ridge


$$
Q(\boldsymbol{\beta})=\frac{1}{n}(\mathbf{y}-\mathbf{X} \boldsymbol{\beta})^{\prime}(\mathbf{y}-\mathbf{X} \boldsymbol{\beta})+ \lambda ||{\beta}||_2
$$

$$
\hat \beta = \left(X^\top X +n \lambda \mathbf{I} \right)^{-1} X^{\top} Y
$$

So the 
$$
\hat Y = H Y = X \left(X^\top X + n\lambda \mathbf{I} \right)^{-1} X^{\top} Y
$$


##### Training and testing error of Ridge

In [42]:
n_tr,p = X_tr.shape

nlamb = 10
beta_without_y = u.inv(X_tr.T @ X_tr + nlamb * np.identity(p)) @ X_tr.T
beta_hat_ridge = beta_without_y @ y_tr
H_tr_ridge = X_tr @ beta_without_y
H_te_ridge = X_te @ beta_without_y
y_tr_hat_ridge = H_tr_ridge @ y_tr
y_te_hat_ridge = H_te_ridge @ y_tr 

tr_error_ridge = 1/n_tr * np.sum((y_tr-y_tr_hat_ridge) ** 2)
te_error_ridge = 1/n_te * np.sum((y_te-y_te_hat_ridge) ** 2)

Correction_ridge = 2 * (1/n_tr * np.trace(H_tr_ridge @ V) - 1/n_te * np.trace(H_te_ridge @ V_tr_te))
tr_error_ridge_c = tr_error_ridge + Correction_lmm
print('training error of ridge is {:.4f}, corrected training error is {:.4f}, test error is {:.4f}'.
    format(tr_error_ridge, tr_error_ridge_c, te_error_ridge))

training error of ridge is 11649.9015, corrected training error is 12202.3988, test error is 16695.3414


##### LOO Cross-valication 

In [43]:
n_tr,p = X_tr.shape
H_cv_ridge = np.zeros([n_tr,n_tr])

lamb = 100 
for i in range(n_tr):
    indices_mi= list(range(n_tr))
    indices_mi.remove(i)
    X_minusi = X_tr[indices_mi, :]
    inverse = u.inv(X_minusi.T @  X_minusi + lamb * np.diag(np.ones(p)) )

    temp = X_tr[i,:] @ inverse @ X_minusi.T  
    H_cv_ridge[i, indices_mi] = temp

np.savez('Parameters/H_cv ridge.npz', H_cv_ridge = H_cv_ridge)

KeyboardInterrupt: 

In [142]:
Error_cv_ridge = 1/n_tr * (np.sum((y_tr - H_cv_ridge @ y_tr)**2))

H_te_ridge = X_te @ u.inv(X_tr.T @ X_tr + lamb * np.identity(p)) @ X_tr.T
Error_te_ridge = 1/n_te * (np.sum((y_te - H_te_ridge @ y_tr)**2))


Correction_ridge = 2  * (np.trace(H_cv_ridge @ V)/n_tr - np.trace(H_te_ridge @ V_tr_te) /n_te)
Error_cv_ridge_c = Error_cv_ridge + Correction_ridge

print('training error of ridge is {:.4f}, corrected training error is {:.4f}, test error is {:.4f}'.
    format(Error_cv_ridge, Error_cv_ridge_c, Error_te_ridge))

training error of ridge is 0.8204, corrected training error is 0.9965, test error is 0.8893


##### CV with k folds

In [44]:
@timing
def H_function_ridge(X_minus_k, X_k, y_minus_k, y_k ):

    return X_k @ u.inv(X_minus_k.T @ X_minus_k + lamb * np.identity(X_minus_k.shape[1])) @ X_minus_k.T

H_cv_ridge_k = getHcv_for_Kfolds(X_tr, y_tr, H_function_ridge, nfolds = 10)

First 5 fold indices : [(0, 204), (204, 408), (408, 612), (612, 816), (816, 1020)]
Calculating fold #  0
--- start function H_function_ridge ---
------ 0.8700 seconds -----
Calculating fold #  1
--- start function H_function_ridge ---
------ 0.8705 seconds -----
Calculating fold #  2
--- start function H_function_ridge ---
------ 0.8790 seconds -----
Calculating fold #  3
--- start function H_function_ridge ---
------ 0.8870 seconds -----
Calculating fold #  4
--- start function H_function_ridge ---
------ 0.9060 seconds -----
Calculating fold #  5
--- start function H_function_ridge ---
------ 0.8620 seconds -----
Calculating fold #  6
--- start function H_function_ridge ---
------ 0.8730 seconds -----
Calculating fold #  7
--- start function H_function_ridge ---
------ 0.8900 seconds -----
Calculating fold #  8
--- start function H_function_ridge ---
------ 0.9540 seconds -----
Calculating fold #  9
--- start function H_function_ridge ---
------ 0.8660 seconds -----


In [45]:
V = fast.V()
V_tr_te = 1/sc * fast.sigma_g2* W_tr @ W_te.T
Error_cv_ridge = 1/n_tr * (np.sum((y_tr - H_cv_ridge_k @ y_tr)**2))

p = X_tr.shape[1]
H_te_ridge = X_te @ u.inv(X_tr.T @ X_tr + lamb * np.identity(p)) @ X_tr.T
Error_te_ridge = 1/n_te * (np.sum((y_te - H_te_ridge @ y_tr)**2))


Correction_ridge = 2  * (1/n_tr*np.trace(H_cv_ridge_k @ V) - 1/n_te * np.trace(H_te_ridge @ V_tr_te))
Error_cv_ridge_c = Error_cv_ridge + Correction_ridge

print('The CV error and CVc error and test error of ridge are {:.4f}, {:.4f}, {:.4f}'
.format(Error_cv_ridge, Error_cv_ridge_c, Error_te_ridge))

The CV error and CVc error and test error of ridge are 14618.3514, 14753.0559, 15561.9900


Test the model with random generated data `X_te_random` and `y_te_random`

In [46]:
V_inv = fast.V_inv()
# using estimated sigma_g2 to get the Covariance(y_tr, y_te)
V_tr_te = 1/sc * fast.sigma_g2 * W_tr @ W_te_random.T

# using all 2000 snps with the intercept terms
X_te_random_all = np.concatenate([X_te_random[:,[0]], W_te_random[:,:]], axis = 1)
H_te_ridge = X_te_random_all @ beta_without_y 
y_te_random_hat_ridge = H_te_ridge @ y_tr


V = fast.V()
Correction_ridge = 2 * (1/n_tr * np.trace(H_tr_ridge @ V) - 1/ n_te * np.trace(H_te_ridge @ V_tr_te))
tr_error_ridge_c = tr_error_ridge + Correction_ridge

te_error_ridge_all = 1/n_te * np.sum((y_te_random - y_te_random_hat_ridge) ** 2)

print('training error of ridge is {:.4f}, corrected training error is {:.4f}, test error is {:.4f}'.
    format(tr_error_ridge, tr_error_ridge_c, te_error_ridge_all))

NameError: name 'W_te_random' is not defined

##### Using K-folds cross-validation

In [61]:
lamb = 100
@timing
def H_function_ridge(X_minus_k, X_k, y_minus_k, y_k ):
    return X_k @ u.inv(X_minus_k.T @ X_minus_k + lamb * np.identity(X_minus_k.shape[1])) @ X_minus_k.T

H_cv_ridge_all_k = getHcv_for_Kfolds(X_tr_all, y_tr, H_function_ridge, nfolds = 10)

NameError: name 'X_tr_all' is not defined

In [176]:
V = fast.V()
V_tr_te = 1/sc * fast.sigma_g2* W_tr @ W_te.T
Error_cv_ridge = 1/n_tr * (np.sum((y_tr - H_cv_ridge_all_k @ y_tr)**2))

p = X_tr_all.shape[1]
# H_te_ridge = X_te_all @ u.inv(X_tr_all.T @ X_tr_all + lamb * np.identity(p)) @ X_tr_all.T
H_te_ridge = X_te_all @ beta_without_y
Error_te_ridge = 1/n_te * (np.sum((y_te - H_te_ridge @ y_tr)**2))


Correction_ridge = 2  * (1/n_tr*np.trace(H_cv_ridge_all_k @ V) - 1/n_te * np.trace(H_te_ridge @ V_tr_te))
Error_cv_ridge_c = Error_cv_ridge + Correction_ridge

print('The CV error and CVc error and test error of ridge are {:.4f}, {:.4f}, {:.4f}'
.format(Error_cv_ridge, Error_cv_ridge_c, Error_te_ridge))

The CV error and CVc error and test error of ridge are 0.6286, 0.6267, 0.7724


In [171]:
# using theoretical sigma
# V = fast.V(sigma_g2=sigma_g2_test, sigma_e2 = sigma_e2)
# V =  1/sc * sigma_g2_test * W_tr @ W_tr.T + sigma_e2 * np.identity(X_tr.shape[0])
V = fast.V()
V_tr_te = 1/sc * fast.sigma_g2* W_tr@ W_te_random.T
Error_cv_ridge = 1/n_tr * (np.sum((y_tr - H_cv_ridge_k @ y_tr)**2))

H_te_ridge = X_te_all @ u.inv(X_tr_all.T @ X_tr_all + lamb * np.identity(p)) @ X_tr_all.T
Error_te_ridge = 1/n_te * (np.sum((y_te - H_te_ridge @ y_tr)**2))


Correction_ridge = 2  * (1/n_tr*np.trace(H_cv_ridge_k @ V) - 1/n_te * np.trace(H_te_ridge @ V_tr_te))
Error_cv_ridge_c = Error_cv_ridge + Correction_ridge

print(Correction_ridge)
print('The CV error and CVc error and test error of ridge are {:.2f}, {:.2f}, {:.2f}'
.format(Error_cv_ridge, Error_cv_ridge_c, Error_te_ridge))

0.16750499612001546
The CV error and CVc error and test error of ridge are 0.82, 0.99, 0.64


##### LOO Cross-valication 

In [56]:
n_tr,p = X_tr_all.shape
H_cv_ridge = np.zeros([n_tr,n_tr])

lamb = 100 
for i in range(n_tr):
    indices_mi= list(range(n_tr))
    indices_mi.remove(i)
    X_minusi = X_tr_all[indices_mi, :]
    inverse = u.inv(X_minusi.T @  X_minusi + lamb * np.diag(np.ones(p)) )

    temp = X_tr_all[i,:] @ inverse @ X_minusi.T  
    H_cv_ridge[i, indices_mi] = temp

np.savez('Parameters/H_cv ridge.npz', H_cv_ridge = H_cv_ridge)

In [ ]:
Error_cv_ridge = 1/n_tr * (np.sum((y_tr - H_cv_ridge @ y_tr)**2))

H_te_ridge = X_te @ u.inv(X_tr.T @ X_tr + lamb * np.identity(p)) @ X_tr.T
Error_te_ridge = 1/n_te * (np.sum((y_te - H_te_ridge @ y_tr)**2))


Correction_ridge = 2  * (np.trace(H_cv_ridge @ V)/n_tr - np.trace(H_te_ridge @ V_tr_te) /n_te)
Error_cv_ridge_c = Error_cv_ridge + Correction_ridge

print('The CV error and CVc error and test error of ridge are {:.2f}, {:.2f}, {:.2f}'
.format(Error_cv_ridge, Error_cv_ridge_c, Error_te_ridge))

The CV error and CVc error and test error of ridge are 3971.16, 4005.64, 4310.60
